Preprocessing Anime dataset from Kaggle

In [272]:
from collections import defaultdict
import pandas as pd
import numpy as np
import sklearn as sk
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import mean_absolute_error
import matplotlib.pyplot as plt
import re
import string
import itertools

1. Data preprocessing

1.1 Data Cleaning

Anime data

In [273]:
# remove unwanted features (columns) from the dataset
anime_df = pd.read_csv("animes.csv")
anime_df.rename(columns={'title': 'name'}, inplace=True)
anime_df.drop(['aired', 'ranked', 'img_url', 'link'], axis=1, inplace=True)

#removing unwanted characters from the anime name strings
def text_cleaning(text):
    text = re.sub(r'&quot;', '', text)
    text  = "".join([char for char in text if char not in string.punctuation])
    text = re.sub(r'.hack//', '', text)
    text = re.sub(r'&#039;', '', text)
    text = re.sub(r'A&#039;s', '', text)
    text = re.sub(r'I&#039;', 'I\'', text)
    text = re.sub(r'&amp;', 'and', text)
    text = re.sub(r'Â°', '',text)

    return text

anime_df['name'] = anime_df['name'].apply(text_cleaning)
anime_df.head(5)

,uid,name,synopsis,genre,episodes,members,popularity,score
0,28891,Haikyuu Second Season,Following their participation at the Inter-Hig...,"['Comedy', 'Sports', 'Drama', 'School', 'Shoun...",25.0,489888,141,8.82
1,23273,Shigatsu wa Kimi no Uso,Music accompanies the path of the human metron...,"['Drama', 'Music', 'Romance', 'School', 'Shoun...",22.0,995473,28,8.83
2,34599,Made in Abyss,The Abyss—a gaping chasm stretching down into ...,"['Sci-Fi', 'Adventure', 'Mystery', 'Drama', 'F...",13.0,581663,98,8.83
3,5114,Fullmetal Alchemist Brotherhood,"""In order for something to be obtained, someth...","['Action', 'Military', 'Adventure', 'Comedy', ...",64.0,1615084,4,9.23
4,31758,Kizumonogatari III Reiketsuhen,After helping revive the legendary vampire Kis...,"['Action', 'Mystery', 'Supernatural', 'Vampire']",1.0,214621,502,8.83


In [274]:
anime_df.rename(columns={'uid': 'anime_id', 'score': 'rating'}, inplace=True)
anime_df.episodes.replace({'Unknown':np.nan},inplace=True)

anime_df.drop_duplicates(subset=['name'], inplace=True)
anime_df.dropna(inplace=True)
anime_df.reset_index(drop=True, inplace=True)

anime_df.head(5)

,anime_id,name,synopsis,genre,episodes,members,popularity,rating
0,28891,Haikyuu Second Season,Following their participation at the Inter-Hig...,"['Comedy', 'Sports', 'Drama', 'School', 'Shoun...",25.0,489888,141,8.82
1,23273,Shigatsu wa Kimi no Uso,Music accompanies the path of the human metron...,"['Drama', 'Music', 'Romance', 'School', 'Shoun...",22.0,995473,28,8.83
2,34599,Made in Abyss,The Abyss—a gaping chasm stretching down into ...,"['Sci-Fi', 'Adventure', 'Mystery', 'Drama', 'F...",13.0,581663,98,8.83
3,5114,Fullmetal Alchemist Brotherhood,"""In order for something to be obtained, someth...","['Action', 'Military', 'Adventure', 'Comedy', ...",64.0,1615084,4,9.23
4,31758,Kizumonogatari III Reiketsuhen,After helping revive the legendary vampire Kis...,"['Action', 'Mystery', 'Supernatural', 'Vampire']",1.0,214621,502,8.83


In [275]:
# replace the characters "[]'" with an empty space as the genre column is already of type string
anime_df['genre'] = anime_df['genre'].str.replace("'", "", regex=False)
anime_df['genre'] = anime_df['genre'].str.replace("[", "", regex=False)
anime_df['genre'] = anime_df['genre'].str.replace("]", "", regex=False)

anime_df.head(5)

,anime_id,name,synopsis,genre,episodes,members,popularity,rating
0,28891,Haikyuu Second Season,Following their participation at the Inter-Hig...,"Comedy, Sports, Drama, School, Shounen",25.0,489888,141,8.82
1,23273,Shigatsu wa Kimi no Uso,Music accompanies the path of the human metron...,"Drama, Music, Romance, School, Shounen",22.0,995473,28,8.83
2,34599,Made in Abyss,The Abyss—a gaping chasm stretching down into ...,"Sci-Fi, Adventure, Mystery, Drama, Fantasy",13.0,581663,98,8.83
3,5114,Fullmetal Alchemist Brotherhood,"""In order for something to be obtained, someth...","Action, Military, Adventure, Comedy, Drama, Ma...",64.0,1615084,4,9.23
4,31758,Kizumonogatari III Reiketsuhen,After helping revive the legendary vampire Kis...,"Action, Mystery, Supernatural, Vampire",1.0,214621,502,8.83


Ratings data

In [289]:
user_ratings_df = pd.read_csv("reviews.csv")
user_ratings_df.drop(['link', 'text'], axis=1, inplace=True)
user_ratings_df.rename(columns={'profile': 'user_id'}, inplace=True)

user_ratings_df.user_id = pd.factorize(user_ratings_df.user_id)[0]

user_ratings_df.head()

,uid,user_id,anime_uid,score,scores
0,255938,0,34096,8,"{'Overall': '8', 'Story': '8', 'Animation': '8..."
1,259117,1,34599,10,"{'Overall': '10', 'Story': '10', 'Animation': ..."
2,253664,2,28891,7,"{'Overall': '7', 'Story': '7', 'Animation': '9..."
3,8254,3,2904,9,"{'Overall': '9', 'Story': '9', 'Animation': '9..."
4,291149,4,4181,10,"{'Overall': '10', 'Story': '10', 'Animation': ..."


In [290]:
user_ratings_df['scores'] = user_ratings_df['scores'].str.replace("'", "", regex=False)
user_ratings_df['scores'] = user_ratings_df['scores'].str.replace("{", "", regex=False)
user_ratings_df['scores'] = user_ratings_df['scores'].str.replace("}", "", regex=False)

user_ratings_df.head()

,uid,user_id,anime_uid,score,scores
0,255938,0,34096,8,"Overall: 8, Story: 8, Animation: 8, Sound: 10,..."
1,259117,1,34599,10,"Overall: 10, Story: 10, Animation: 10, Sound: ..."
2,253664,2,28891,7,"Overall: 7, Story: 7, Animation: 9, Sound: 8, ..."
3,8254,3,2904,9,"Overall: 9, Story: 9, Animation: 9, Sound: 10,..."
4,291149,4,4181,10,"Overall: 10, Story: 10, Animation: 8, Sound: 9..."


In [291]:
user_ratings_df['scores'] = [re.sub("[^0-9,]", "", anime) for anime in user_ratings_df['scores']]

user_ratings_df.head()

,uid,user_id,anime_uid,score,scores
0,255938,0,34096,8,"8,8,8,10,9,8"
1,259117,1,34599,10,"10,10,10,10,10,10"
2,253664,2,28891,7,"7,7,9,8,8,8"
3,8254,3,2904,9,"9,9,9,10,10,9"
4,291149,4,4181,10,"10,10,8,9,10,10"


In [292]:
category_ratings_df = user_ratings_df['scores'].str.split(",", expand=True)
category_ratings_df.columns = ['Overall', 'Story', 'Animation','Sound', 'Character', 'Enjoyment']

category_ratings_df.head(10)

,Overall,Story,Animation,Sound,Character,Enjoyment
0,8,8,8,10,9,8
1,10,10,10,10,10,10
2,7,7,9,8,8,8
3,9,9,9,10,10,9
4,10,10,8,9,10,10
5,10,9,10,10,10,10
6,6,6,9,8,6,6
7,8,7,9,0,0,9
8,10,10,9,10,10,10
9,4,8,9,8,5,4


Finalised user_ratings_df

In [294]:
user_ratings_df = pd.concat([user_ratings_df, category_ratings_df], axis=1)
user_ratings_df.drop(columns=['score', 'scores', 'uid'], inplace=True)
user_ratings_df.rename(columns={"anime_uid": "anime_id"}, inplace=True)

user_ratings_df.head()

,user_id,anime_id,Overall,Story,Animation,Sound,Character,Enjoyment
0,0,34096,8,8,8,10,9,8
1,1,34599,10,10,10,10,10,10
2,2,28891,7,7,9,8,8,8
3,3,2904,9,9,9,10,10,9
4,4,4181,10,10,8,9,10,10


In [295]:
user_ratings_df[['Overall', 'Story', 'Animation', 'Sound', 'Character', 'Enjoyment']] = user_ratings_df[
    ['Overall', 'Story', 'Animation', 'Sound', 'Character', 'Enjoyment']].apply(pd.to_numeric)
    

testing to see if a user has multiple reviews for 

In [297]:
user_ratings_df['user_id'].unique()

array([    0,     1,     2, ..., 47882, 47883, 47884], dtype=int64)

In [298]:
user_0 = user_ratings_df[user_ratings_df['user_id'] == 0]

user_0.head()

,user_id,anime_id,Overall,Story,Animation,Sound,Character,Enjoyment
0,0,34096,8,8,8,10,9,8
2089,0,12403,6,6,9,7,7,5
3170,0,35073,6,6,8,7,7,6
3658,0,23225,7,8,9,7,7,6
4306,0,2035,8,9,7,8,9,8


In [371]:
# Group the ratings dataframe by user ID and count the number of ratings per user
user_ratings_count = user_ratings_df.groupby('user_id')['Overall'].count()

print(user_ratings_count)

user_id
0        148
1         25
2         20
3          9
4          5
        ... 
47880      1
47881      1
47882      1
47883      1
47884      1
Name: Overall, Length: 47885, dtype: int64


In [312]:
# user_0 = user_ratings_df[user_ratings_df['user_id'] == 0]

# user_0.head()

,user_id,anime_id,Overall,Story,Animation,Sound,Character,Enjoyment
0,0,34096,8,8,8,10,9,8
2089,0,12403,6,6,9,7,7,5
3170,0,35073,6,6,8,7,7,6
3658,0,23225,7,8,9,7,7,6
4306,0,2035,8,9,7,8,9,8


In [414]:
# def get_normalised_rating(user_id):
#     # mean, std = get_mean_and_std(user_id)

#     # user = user_ratings_df[user_ratings_df['user_id'] == user_id]
    
#     # user_norm_ratings = user.apply(lambda row: ((row['Overall'] - mean) / std), axis=1)

#     # return user_norm_ratings.mean()

#     return mae_df.groupby('user_id')['normalized_rating'].mean()

In [415]:
# user_mean_std_df['normalised'] = [get_normalised_rating(user_id) for user_id in 
# user_ratings_df['user_id'].unique()]

KeyboardInterrupt: 

In [ ]:
user_mean_std_df.head()

,mean,std
0,7.324324,1.346278
1,9.040000,0.978093
2,6.700000,1.218282
3,8.888889,0.927961
4,6.000000,3.674235


USE MERGED DATASET FOR BOTH CB AND CF RECOMMENDATION METHODS TO PREVENT THE INDEX ERROR OCCURING IN THE HYBRID METHOD -> SOME ANIMES ARE REMOVED AFTER MERGING SO LEADS TO INDEX ERROR

In [10]:
anime_with_ratings_df = pd.merge(anime_df, user_ratings_df, on='anime_id')

anime_with_ratings_df.drop_duplicates(subset=['user_id', 'name'], inplace=True)
anime_with_ratings_df.reset_index(drop=True, inplace=True)

anime_with_ratings_df.head()

,anime_id,name,synopsis,genre,episodes,members,popularity,rating,user_id,Overall,Story,Animation,Sound,Character,Enjoyment
0,28891,Haikyuu Second Season,Following their participation at the Inter-Hig...,"Comedy, Sports, Drama, School, Shounen",25.0,489888,141,8.82,2,7,7,9,8,8,8
1,28891,Haikyuu Second Season,Following their participation at the Inter-Hig...,"Comedy, Sports, Drama, School, Shounen",25.0,489888,141,8.82,15102,10,10,9,9,10,10
2,28891,Haikyuu Second Season,Following their participation at the Inter-Hig...,"Comedy, Sports, Drama, School, Shounen",25.0,489888,141,8.82,15103,10,10,9,9,10,10
3,28891,Haikyuu Second Season,Following their participation at the Inter-Hig...,"Comedy, Sports, Drama, School, Shounen",25.0,489888,141,8.82,13930,7,7,7,9,0,6
4,28891,Haikyuu Second Season,Following their participation at the Inter-Hig...,"Comedy, Sports, Drama, School, Shounen",25.0,489888,141,8.82,599,8,7,7,9,8,9


Content based filtering recommendation

USE IN FINAL REPORT:

Normalizing ratings is important in content-based recommenders for anime (and any other system) for several reasons:

Fairness: Different users may have different rating scales. For example, one user may only give a rating of 4 or 5 if they really loved the anime, while another user may give a rating of 3 if they thought it was just okay. Normalizing the ratings ensures that each user's ratings are treated equally and fairly, regardless of their personal rating scale.

Consistency: Normalizing the ratings ensures that they all fall within the same range, making it easier to compare the ratings of different anime. This helps to ensure that the recommendations are consistent and reliable.

Accuracy: Normalizing the ratings can help to reduce the impact of outliers, which can skew the results of the recommender system. By scaling the ratings to a common range, outliers can be identified and handled appropriately, leading to more accurate recommendations.

In summary, normalizing ratings in a content-based recommender for anime helps to ensure fairness, consistency, and accuracy in the recommendations provided to users.

In [11]:
normalised_anime_df = anime_df.copy()

weights = {
    'genre': 0.35,
    'members_norm': 0.1,
    'rating_norm': 0.35,
    'popularity_norm': 0.1,
    'episodes_norm': 0.1
}

normalised_anime_df['members_norm'] = normalised_anime_df['members'] / normalised_anime_df['members'].max() * weights['members_norm']
normalised_anime_df['avg_rating_norm'] = normalised_anime_df['rating'] / normalised_anime_df['rating'].max() * weights['rating_norm']
normalised_anime_df['popularity_norm'] = normalised_anime_df['popularity'] / normalised_anime_df['popularity'].max() * weights['popularity_norm']
normalised_anime_df['episodes_norm'] = normalised_anime_df['episodes'] / normalised_anime_df['episodes'].max() * weights['episodes_norm']

normalised_anime_df.head()

,anime_id,name,synopsis,genre,episodes,members,popularity,rating,members_norm,avg_rating_norm,popularity_norm,episodes_norm
0,28891,Haikyuu Second Season,Following their participation at the Inter-Hig...,"Comedy, Sports, Drama, School, Shounen",25.0,489888,141,8.82,0.026183,0.334453,0.000864,0.000818
1,23273,Shigatsu wa Kimi no Uso,Music accompanies the path of the human metron...,"Drama, Music, Romance, School, Shounen",22.0,995473,28,8.83,0.053204,0.334832,0.000172,0.000720
2,34599,Made in Abyss,The Abyss—a gaping chasm stretching down into ...,"Sci-Fi, Adventure, Mystery, Drama, Fantasy",13.0,581663,98,8.83,0.031088,0.334832,0.000600,0.000425
3,5114,Fullmetal Alchemist Brotherhood,"""In order for something to be obtained, someth...","Action, Military, Adventure, Comedy, Drama, Ma...",64.0,1615084,4,9.23,0.086320,0.350000,0.000025,0.002094
4,31758,Kizumonogatari III Reiketsuhen,After helping revive the legendary vampire Kis...,"Action, Mystery, Supernatural, Vampire",1.0,214621,502,8.83,0.011471,0.334832,0.003076,0.000033


In [12]:
normalised_anime_df.drop(['members', 'rating', 'popularity', 'episodes'], axis=1, inplace=True)

normalised_anime_df.head()

,anime_id,name,synopsis,genre,members_norm,avg_rating_norm,popularity_norm,episodes_norm
0,28891,Haikyuu Second Season,Following their participation at the Inter-Hig...,"Comedy, Sports, Drama, School, Shounen",0.026183,0.334453,0.000864,0.000818
1,23273,Shigatsu wa Kimi no Uso,Music accompanies the path of the human metron...,"Drama, Music, Romance, School, Shounen",0.053204,0.334832,0.000172,0.000720
2,34599,Made in Abyss,The Abyss—a gaping chasm stretching down into ...,"Sci-Fi, Adventure, Mystery, Drama, Fantasy",0.031088,0.334832,0.000600,0.000425
3,5114,Fullmetal Alchemist Brotherhood,"""In order for something to be obtained, someth...","Action, Military, Adventure, Comedy, Drama, Ma...",0.086320,0.350000,0.000025,0.002094
4,31758,Kizumonogatari III Reiketsuhen,After helping revive the legendary vampire Kis...,"Action, Mystery, Supernatural, Vampire",0.011471,0.334832,0.003076,0.000033


In [13]:
genres_df = anime_df['genre'].str.get_dummies(sep=', ').astype(int)
genres_df = genres_df.apply(lambda x : x * weights['genre'])

genres_df.head()

,Action,Adventure,Cars,Comedy,Dementia,Demons,Drama,Ecchi,Fantasy,Game,...,Shounen Ai,Slice of Life,Space,Sports,Super Power,Supernatural,Thriller,Vampire,Yaoi,Yuri
0,0.00,0.00,0.0,0.35,0.0,0.0,0.35,0.0,0.00,0.0,...,0.0,0.0,0.0,0.35,0.0,0.00,0.0,0.00,0.0,0.0
1,0.00,0.00,0.0,0.00,0.0,0.0,0.35,0.0,0.00,0.0,...,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.0
2,0.00,0.35,0.0,0.00,0.0,0.0,0.35,0.0,0.35,0.0,...,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.0
3,0.35,0.35,0.0,0.35,0.0,0.0,0.35,0.0,0.35,0.0,...,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.0
4,0.35,0.00,0.0,0.00,0.0,0.0,0.00,0.0,0.00,0.0,...,0.0,0.0,0.0,0.00,0.0,0.35,0.0,0.35,0.0,0.0


In [14]:
normalised_anime_df.drop('genre', axis=1, inplace=True)
normalised_anime_df = pd.concat([normalised_anime_df, genres_df], axis=1)

normalised_anime_df.head()

,anime_id,name,synopsis,members_norm,avg_rating_norm,popularity_norm,episodes_norm,Action,Adventure,Cars,...,Shounen Ai,Slice of Life,Space,Sports,Super Power,Supernatural,Thriller,Vampire,Yaoi,Yuri
0,28891,Haikyuu Second Season,Following their participation at the Inter-Hig...,0.026183,0.334453,0.000864,0.000818,0.00,0.00,0.0,...,0.0,0.0,0.0,0.35,0.0,0.00,0.0,0.00,0.0,0.0
1,23273,Shigatsu wa Kimi no Uso,Music accompanies the path of the human metron...,0.053204,0.334832,0.000172,0.000720,0.00,0.00,0.0,...,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.0
2,34599,Made in Abyss,The Abyss—a gaping chasm stretching down into ...,0.031088,0.334832,0.000600,0.000425,0.00,0.35,0.0,...,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.0
3,5114,Fullmetal Alchemist Brotherhood,"""In order for something to be obtained, someth...",0.086320,0.350000,0.000025,0.002094,0.35,0.35,0.0,...,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.0
4,31758,Kizumonogatari III Reiketsuhen,After helping revive the legendary vampire Kis...,0.011471,0.334832,0.003076,0.000033,0.35,0.00,0.0,...,0.0,0.0,0.0,0.00,0.0,0.35,0.0,0.35,0.0,0.0


In [15]:
features = ['members_norm', 'avg_rating_norm', 'popularity_norm', 'episodes_norm'] + genres_df.columns.tolist()

cosine_sim = cosine_similarity(normalised_anime_df[features], normalised_anime_df[features])

print(cosine_sim)

[[1.         0.66214591 0.32441239 ... 0.38178832 0.24895614 0.65314481]
 [0.66214591 1.         0.32520761 ... 0.16702495 0.24895703 0.65247583]
 [0.32441239 0.32520761 1.         ... 0.16711943 0.24916537 0.30803338]
 ...
 [0.38178832 0.16702495 0.16711943 ... 1.         0.26943555 0.58509904]
 [0.24895614 0.24895703 0.24916537 ... 0.26943555 1.         0.21806531]
 [0.65314481 0.65247583 0.30803338 ... 0.58509904 0.21806531 1.        ]]


In [16]:
indices = pd.Series(anime_df.index, index=anime_df['name']).drop_duplicates()

Recommend one season of a show.

E.g. if the recommendations have "Tokyo ghoul season 1, tokyo ghoul season 2" etc. we want to only recommend one of these.

Recommend the one with the highest average rating

In [17]:
def get_unique_recommendations(anime_titles, anime_df=anime_df):
    titles = anime_titles[:]

    # sort all the anime titles so that it is easier to group similar titles
    titles.sort()
    iterator = itertools.groupby(titles, lambda string: string.split(' ')[0])

    grouped_titles = []
    for element, group in iterator:
        grouped_titles.append(list(group))

    # checking for each grouped anime title which one has the highest average rating so that we can recommend
    # that one to the user
    unique_titles = []
    title = ''
    for anime_group in grouped_titles:
        max_rating = 0
        for anime in anime_group:
            anime_index = anime_df[anime_df['name'] == anime].index
            curr_rating = anime_df['rating'].iloc[anime_index[0]]

            if curr_rating > max_rating:
                max_rating = anime_df['rating'].iloc[anime_index[0]]
                title = anime_df['name'].iloc[anime_index[0]]

        unique_titles.append(title)

    return unique_titles

In [60]:
def content_based_recommendations(title, cosine_sim=cosine_sim, anime_df=anime_df, indices=indices, n_recommendations=100):
    # Get the index of the anime that matches the title
    index = indices[title]
    
    # Get the pairwise cosine similarity scores for all anime with that index
    sim_scores = list(enumerate(cosine_sim[index]))

    # Sort the anime based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the top 100 most similar anime -> allows us to have more anime so that we are 
    # still able to recommend n_recommendations animes to the user after getting all the unique titles
    sim_scores = sim_scores[1:101]

    # Get the titles of the top 10 most similar anime
    anime_indices = [i[0] for i in sim_scores]
    
    anime_titles = anime_df['name'].iloc[anime_indices].values.tolist()

    # convert to set for constant lookup time
    unique_titles = set(get_unique_recommendations(anime_titles))

    recommendations = [i for i in anime_titles if i in unique_titles]

    return recommendations[:n_recommendations+1]

In [19]:
content_based_recommendations('Death Note')

['Mirai Nikki',
 'B The Beginning',
 'Higurashi no Naku Koro ni Kai',
 'Mousou Dairinin',
 'Imawa no Kuni no Alice OVA',
 'Mouryou no Hako',
 'Zankyou no Terror',
 'Babylon',
 'Mi Yu Xing Zhe',
 'Yakushiji Ryouko no Kaiki Jikenbo',
 'Saint Luminous Jogakuin',
 'Yakusoku no Neverland',
 'Boku dake ga Inai Machi',
 'Detective Conan Movie 06 The Phantom of Baker Street',
 'Jigoku Shoujo Futakomori',
 'Black Jack',
 'Monster',
 'Shinreigari',
 'Umineko no Naku Koro ni',
 'Kishibe Rohan wa Ugokanai',
 'Karakurizoushi Ayatsuri Sakon',
 'ChäoSChild Silent Sky',
 'Kagewani Shou',
 'Madlax',
 'Kindaichi Shounen no Jikenbo',
 'UNGO Ingaron',
 'Totsukuni no Shoujo',
 'Shuangsheng Lingtan',
 'Tokyo Ravens Kon ga Oshiete Moshi Agemasu Maru Wakari Tokyo Ravens',
 '18if',
 'Oshie to Tabi Suru Otoko',
 'Bloody Night',
 'Death Parade',
 'Mahou Shoujo Madoka★Magica Movie 3 Hangyaku no Monogatari',
 'Kara no Kyoukai 7 Satsujin Kousatsu Kou',
 'Another',
 'xxxHOLiC Kei',
 'AD Police',
 'Gankutsuou',
 'Shi

Item-item Collaborative filtering

From the recommendations provided from the content based algorithm above, calculate the similarity of each of those recommendations with anime the user has already watched.

maybe select users that have rated more than a certain threshold of anime? E.g. rated more than 200 animes

In [20]:
user_ratings_df['user_id'].value_counts()

2762     762
456      743
109      648
97       566
31       491
        ... 
36472      1
36473      1
36476      1
36481      1
47884      1
Name: user_id, Length: 47885, dtype: int64

Create User-item matrix for item-item collaborative filtering

In [21]:
anime_with_ratings_df.head()

,anime_id,name,synopsis,genre,episodes,members,popularity,rating,user_id,Overall,Story,Animation,Sound,Character,Enjoyment
0,28891,Haikyuu Second Season,Following their participation at the Inter-Hig...,"Comedy, Sports, Drama, School, Shounen",25.0,489888,141,8.82,2,7,7,9,8,8,8
1,28891,Haikyuu Second Season,Following their participation at the Inter-Hig...,"Comedy, Sports, Drama, School, Shounen",25.0,489888,141,8.82,15102,10,10,9,9,10,10
2,28891,Haikyuu Second Season,Following their participation at the Inter-Hig...,"Comedy, Sports, Drama, School, Shounen",25.0,489888,141,8.82,15103,10,10,9,9,10,10
3,28891,Haikyuu Second Season,Following their participation at the Inter-Hig...,"Comedy, Sports, Drama, School, Shounen",25.0,489888,141,8.82,13930,7,7,7,9,0,6
4,28891,Haikyuu Second Season,Following their participation at the Inter-Hig...,"Comedy, Sports, Drama, School, Shounen",25.0,489888,141,8.82,599,8,7,7,9,8,9


In [22]:
def create_pivot_table(data, value):
    pivot_table = data.pivot_table(index='user_id', columns='name', values=value)
    pivot_table.fillna(0, inplace=True)

    return pivot_table

Create pivot tables for each rating category (story, animation, etc.)

In [23]:
# approx 2mins to execute
overall_pivot = create_pivot_table(anime_with_ratings_df, 'Overall')
story_pivot = create_pivot_table(anime_with_ratings_df, 'Story')
animation_pivot = create_pivot_table(anime_with_ratings_df, 'Animation')
sound_pivot = create_pivot_table(anime_with_ratings_df, 'Sound')
character_pivot = create_pivot_table(anime_with_ratings_df, 'Character')
enjoyment_pivot = create_pivot_table(anime_with_ratings_df, 'Enjoyment')

overall_pivot.head()

name,0,0008,001,009 ReCyborg,0091,07Ghost,100,100 Pascalsensei TV,1000nen Joou Queen Millennia,1001 Nights,...,makemagic,sCRYed,vivi,xxxHOLiC,xxxHOLiC Kei,xxxHOLiC Movie Manatsu no Yoru no Yume,xxxHOLiC Rou,xxxHOLiC Shunmuki,ēlDLIVE,◯
user_id,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Use Cosine Similarity to calculate the similarity between each anime using the different categories rated by the user.

In [24]:
def calculate_similarities(pivot_table):
    similarities = cosine_similarity(pivot_table.T)
    similarities_df = pd.DataFrame(similarities, index=pivot_table.columns, columns=pivot_table.columns)

    return similarities_df

Create pivot tables to represent each category

In [25]:
overall_similarities_df = calculate_similarities(overall_pivot)
story_similarities_df = calculate_similarities(story_pivot)
animation_similarities_df = calculate_similarities(animation_pivot)
sound_similarities_df = calculate_similarities(sound_pivot)
character_similarities_df = calculate_similarities(character_pivot)
enjoyment_similarities_df = calculate_similarities(enjoyment_pivot)

Combine the pivot tables using the assigned weights to combine the values

In [26]:
overall_weight = 0.5 
story_weight=0.1 
animation_weight=0.1
sound_weight=0.1
character_weight=0.2
enjoyment_weight=0.1

combined_category_ratings_pivot = (overall_similarities_df * overall_weight) + (story_similarities_df * story_weight) + (animation_similarities_df * animation_weight) 
(sound_similarities_df * sound_weight) + (character_similarities_df * character_weight) + (enjoyment_similarities_df * enjoyment_weight)

combined_category_ratings_pivot.head()

name,0,0008,001,009 ReCyborg,0091,07Ghost,100,100 Pascalsensei TV,1000nen Joou Queen Millennia,1001 Nights,...,makemagic,sCRYed,vivi,xxxHOLiC,xxxHOLiC Kei,xxxHOLiC Movie Manatsu no Yoru no Yume,xxxHOLiC Rou,xxxHOLiC Shunmuki,ēlDLIVE,◯
name,,,,,,,,,,,,,,,,,,,,,
0,0.700000,0.162776,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
0008,0.162776,0.700000,0.019296,0.052205,0.026402,0.000000,0.0,0.0,0.0,0.019980,...,0.0,0.0,0.116281,0.0,0.0,0.0,0.0,0.0,0.0,0.040191
001,0.000000,0.019296,0.700000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.019867,...,0.0,0.0,0.116054,0.0,0.0,0.0,0.0,0.0,0.0,0.119088
009 ReCyborg,0.000000,0.052205,0.000000,0.700000,0.000000,0.000000,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
0091,0.000000,0.026402,0.000000,0.000000,0.700000,0.050865,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000


Get collaborative filtering recommendations

In [27]:
def collaborative_filtering_recommendations(anime, n=100):
    similarity_scores = combined_category_ratings_pivot[anime]
    similarity_scores = similarity_scores.sort_values(ascending=False)

    similar_anime = similarity_scores.iloc[1:n+1].index.tolist()

    # remove reoccuring anime titles, e.g. Tokyo Ghoul season 1, Tokyo Ghoul season 2, etc.
    unique_titles = set(get_unique_recommendations(similar_anime))

    recommendations = [i for i in similar_anime if i in unique_titles]

    return recommendations


In [28]:
collaborative_filtering_recommendations('Death Note')

['Fullmetal Alchemist Brotherhood',
 'Code Geass Hangyaku no Lelouch R2',
 'SteinsGate',
 'Sword Art Online',
 'Elfen Lied',
 'Cowboy Bebop',
 'Monster',
 'Deadman Wonderland',
 'Ouran Koukou Host Club',
 'Clannad After Story',
 'Highschool of the Dead',
 'Another',
 'Sen to Chihiro no Kamikakushi',
 'Ansatsu Kyoushitsu',
 'Tengen Toppa Gurren Lagann',
 'Howl no Ugoku Shiro',
 'Kill la Kill',
 'Dragon Ball Z',
 'Death Parade',
 'Trigun',
 'Wolfs Rain',
 'One Punch Man',
 'Mirai Nikki',
 'Kenpuu Denki Berserk',
 'Sora no Otoshimono Project Pink',
 'Mousou Dairinin',
 'Kuroshitsuji',
 'Hotaru no Haka',
 'Yuu☆Yuu☆Hakusho',
 'Shiki',
 'Baccano',
 'Black Lagoon',
 'Final Fantasy VII Last Order',
 'Naruto Shippuuden',
 'Neon Genesis Evangelion The End of Evangelion',
 'Serial Experiments Lain',
 'Tsubasa Shunraiki',
 'PsychoPass',
 'Free',
 'Bleach',
 'Vampire Knight',
 'Ookami to Koushinryou',
 'Onegai☆Twins',
 'Darker than Black Kuro no Keiyakusha',
 'School Days',
 'Boku no Hero Academia'

Hybrid Implementation: Combine content based and collaborative filtering methods to provide recommendations to the user

In [77]:
def combined_recommendations(anime_name, num_recommendations=50, content_weight=0.2, collaborative_weight=0.8):
    if anime_name not in combined_category_ratings_pivot.index:
        return []

    content_based = content_based_recommendations(anime_name)
    collaborative_filtering = collaborative_filtering_recommendations(anime_name)

    # removing anime titles that may no longer exist within our dataframe as some were removed after the initial
    # anime_df and ratings_df dataframes were merged together
    content_based_animes = []

    for i in content_based:
        if i in combined_category_ratings_pivot.index:
            content_based_animes.append(i)

    collaborative_based_animes = []

    for i in collaborative_filtering:
        if i in combined_category_ratings_pivot.index:
            collaborative_based_animes.append(i)

    content_based_scores = combined_category_ratings_pivot.loc[content_based_animes]
    collaborative_filtering_scores = combined_category_ratings_pivot.loc[collaborative_based_animes]

    # create weighted scores for all the animes using both the values generated from content based and collaborative filtering methods
    scores = content_based_scores.mul(content_weight).add(collaborative_filtering_scores.mul(collaborative_weight), fill_value=0)
    
    weighted_scores = scores[anime_name].sort_values(ascending=False)
    return weighted_scores, weighted_scores.head(num_recommendations).index.tolist()

RETURNING 2 THINGS FROM THE COMBINED RECOMMENDATION FUNCTION -> BE SURE TO REVERT IF THIS FLOPS

In [78]:
combined_recommendations('Death Note')[1]

['Monster',
 'Fullmetal Alchemist Brotherhood',
 'Another',
 'Code Geass Hangyaku no Lelouch R2',
 'SteinsGate',
 'Sword Art Online',
 'Elfen Lied',
 'Cowboy Bebop',
 'Deadman Wonderland',
 'Ouran Koukou Host Club',
 'Death Parade',
 'Clannad After Story',
 'Mirai Nikki',
 'Highschool of the Dead',
 'Mousou Dairinin',
 'Sen to Chihiro no Kamikakushi',
 'Shiki',
 'Ansatsu Kyoushitsu',
 'Tengen Toppa Gurren Lagann',
 'Howl no Ugoku Shiro',
 'Kill la Kill',
 'Dragon Ball Z',
 'Trigun',
 'Wolfs Rain',
 'One Punch Man',
 'Kenpuu Denki Berserk',
 'Sora no Otoshimono Project Pink',
 'Higurashi no Naku Koro ni Kai',
 'Gankutsuou',
 'Kuroshitsuji',
 'Hotaru no Haka',
 'Yuu☆Yuu☆Hakusho',
 'Baccano',
 'Black Lagoon',
 'Final Fantasy VII Last Order',
 'Naruto Shippuuden',
 'Neon Genesis Evangelion The End of Evangelion',
 'Serial Experiments Lain',
 'Tsubasa Shunraiki',
 'PsychoPass',
 'Free',
 'Bleach',
 'Vampire Knight',
 'Ookami to Koushinryou',
 'Onegai☆Twins',
 'Darker than Black Kuro no Keiy

Measure Performance

MAE (Mean Absolute Error) 

In [360]:
hybrid_recs = combined_recommendations('Shingeki no Kyojin')[0]

hybrid_recs_df = hybrid_recs.to_frame(name='hybrid_score')

hybrid_recs_df.head()

,hybrid_score
name,
Yuri Seijin Naokosan 2012,0.062974
Pokemon,0.055124
Betterman,0.050604
RoKyuBu SS,0.042480
El Cazador de la Bruja,0.037497


Add anime ids to each of the recommendations so we can merge with anime_with_ratings_df 

In [361]:
anime_indexes = [anime_df[anime_df['name'] == anime_name].index[0] for anime_name in hybrid_recs_df.index]
anime_ids = [anime_df.loc[index, 'anime_id'] for index in anime_indexes]

hybrid_recs_df['anime_id'] = anime_ids

hybrid_recs_df.head()

,hybrid_score,anime_id
name,,
Yuri Seijin Naokosan 2012,0.062974,12437
Pokemon,0.055124,527
Betterman,0.050604,1136
RoKyuBu SS,0.042480,16051
El Cazador de la Bruja,0.037497,2030


Create our mae_df to be used specifically for MAE calculations

In [399]:
mae_df = anime_with_ratings_df.copy()

mae_df = pd.merge(anime_with_ratings_df, hybrid_recs_df, on='anime_id')

# convert datatype from object to int to allow mathematical calculations
mae_df[['Overall', 'Story', 'Animation', 'Sound', 'Character', 'Enjoyment']] = mae_df[
    ['Overall', 'Story', 'Animation', 'Sound', 'Character', 'Enjoyment']].apply(pd.to_numeric)
    
# mae_df['normalized_overall'] = mae_df['Overall'] / mae_df['rating'].max()

mae_df.head()

,anime_id,name,synopsis,genre,episodes,members,popularity,rating,user_id,Overall,Story,Animation,Sound,Character,Enjoyment,hybrid_score
0,5114,Fullmetal Alchemist Brotherhood,"""In order for something to be obtained, someth...","Action, Military, Adventure, Comedy, Drama, Ma...",64.0,1615084,4,9.23,15,10,10,9,10,10,10,0.002958
1,5114,Fullmetal Alchemist Brotherhood,"""In order for something to be obtained, someth...","Action, Military, Adventure, Comedy, Drama, Ma...",64.0,1615084,4,9.23,93,10,10,10,9,9,9,0.002958
2,5114,Fullmetal Alchemist Brotherhood,"""In order for something to be obtained, someth...","Action, Military, Adventure, Comedy, Drama, Ma...",64.0,1615084,4,9.23,2521,10,10,10,10,10,10,0.002958
3,5114,Fullmetal Alchemist Brotherhood,"""In order for something to be obtained, someth...","Action, Military, Adventure, Comedy, Drama, Ma...",64.0,1615084,4,9.23,2522,10,10,10,10,10,10,0.002958
4,5114,Fullmetal Alchemist Brotherhood,"""In order for something to be obtained, someth...","Action, Military, Adventure, Comedy, Drama, Ma...",64.0,1615084,4,9.23,2523,7,5,9,8,9,6,0.002958


Create dataframe to store the mean and standard deviation of all users

In [ ]:
def get_mean_and_std(user_id):
    user = user_ratings_df[user_ratings_df['user_id'] == user_id]

    user_mean = user['Overall'].mean()
    user_std = user['Overall'].std()

    return user_mean, user_std

In [ ]:
# for each user id in user_ratings_df['user_id'].unique(), get the mean 
# and standard deviation for that user

# create a dataframe from the results: user_id as index, mean and std as columns

user_mean_std = [get_mean_and_std(user_id) for user_id in user_ratings_df['user_id'].unique()]

user_mean_std_df = pd.DataFrame(user_mean_std, columns=['mean', 'std'])

user_mean_std_df.head()

Add 'normalized_rating' column, which uses the Overall score a user has given an anime alongside the mean and standard deviation of all the animes a user has rated to normalise.

In [411]:
mae_df['normalized_rating'] = mae_df.apply(lambda row: (row['Overall'] - user_mean_std_df.loc[row['user_id'], 'mean']) / user_mean_std_df.loc[row['user_id'], 'std'], axis=1)

mae_df.head()

C:\Users\Haaris PC\AppData\Local\Temp\ipykernel_24692\3650428434.py:1: RuntimeWarning: invalid value encountered in double_scalars
  mae_df['normalized_rating'] = mae_df.apply(lambda row: (row['Overall'] - user_mean_std_df.loc[row['user_id'], 'mean']) / user_mean_std_df.loc[row['user_id'], 'std'], axis=1)


,anime_id,name,synopsis,genre,episodes,members,popularity,rating,user_id,Overall,Story,Animation,Sound,Character,Enjoyment,hybrid_score,normalized_rating
0,5114,Fullmetal Alchemist Brotherhood,"""In order for something to be obtained, someth...","Action, Military, Adventure, Comedy, Drama, Ma...",64.0,1615084,4,9.23,15,10,10,9,10,10,10,0.002958,1.248084
1,5114,Fullmetal Alchemist Brotherhood,"""In order for something to be obtained, someth...","Action, Military, Adventure, Comedy, Drama, Ma...",64.0,1615084,4,9.23,93,10,10,10,9,9,9,0.002958,1.373879
2,5114,Fullmetal Alchemist Brotherhood,"""In order for something to be obtained, someth...","Action, Military, Adventure, Comedy, Drama, Ma...",64.0,1615084,4,9.23,2521,10,10,10,10,10,10,0.002958,1.367527
3,5114,Fullmetal Alchemist Brotherhood,"""In order for something to be obtained, someth...","Action, Military, Adventure, Comedy, Drama, Ma...",64.0,1615084,4,9.23,2522,10,10,10,10,10,10,0.002958,NaN
4,5114,Fullmetal Alchemist Brotherhood,"""In order for something to be obtained, someth...","Action, Military, Adventure, Comedy, Drama, Ma...",64.0,1615084,4,9.23,2523,7,5,9,8,9,6,0.002958,-0.866025


need to get the mean normalised rating for each user to fill in NaN values

In [406]:
# mae_df['normalized_rating'] = mae_df.groupby('user_id')['normalized_rating'].apply(lambda x: x.fillna(x.mean()))

# mae_df.head()

# NaN values are present as the user has only rated this single anime, so they 
# are not that useful when providing recommendations, so we can simply just fill it with 0.
mae_df['normalized_rating'] = mae_df['normalized_rating'].fillna(0)

mae_df.head()

,anime_id,name,synopsis,genre,episodes,members,popularity,rating,user_id,Overall,Story,Animation,Sound,Character,Enjoyment,hybrid_score,normalized_rating
0,5114,Fullmetal Alchemist Brotherhood,"""In order for something to be obtained, someth...","Action, Military, Adventure, Comedy, Drama, Ma...",64.0,1615084,4,9.23,15,10,10,9,10,10,10,0.002958,1.248084
1,5114,Fullmetal Alchemist Brotherhood,"""In order for something to be obtained, someth...","Action, Military, Adventure, Comedy, Drama, Ma...",64.0,1615084,4,9.23,93,10,10,10,9,9,9,0.002958,1.373879
2,5114,Fullmetal Alchemist Brotherhood,"""In order for something to be obtained, someth...","Action, Military, Adventure, Comedy, Drama, Ma...",64.0,1615084,4,9.23,2521,10,10,10,10,10,10,0.002958,1.367527
3,5114,Fullmetal Alchemist Brotherhood,"""In order for something to be obtained, someth...","Action, Military, Adventure, Comedy, Drama, Ma...",64.0,1615084,4,9.23,2522,10,10,10,10,10,10,0.002958,0.000000
4,5114,Fullmetal Alchemist Brotherhood,"""In order for something to be obtained, someth...","Action, Military, Adventure, Comedy, Drama, Ma...",64.0,1615084,4,9.23,2523,7,5,9,8,9,6,0.002958,-0.866025


Normalise the overall rating score so we can use that in MAE, as cosine similarities were computed based upon normalised values.

Normalising the ratings will take the users mean and standard deviation for all the animes they have rated already.

In [407]:
def calculate_mae(recommendations):
    # Create pivot table with users as rows and recommended anime as columns
    recommendations_pivot = recommendations.pivot_table(index='user_id', columns='anime_id', values='hybrid_score')
    
    # Merge pivot table with actual ratings data to get ratings for recommended anime
    ratings = pd.merge(mae_df, recommendations_pivot.stack().reset_index().rename(columns={0: 'hybrid_score'}), on=['user_id', 'anime_id'])
    ratings.drop(columns={'hybrid_score_x'}, inplace=True)
    ratings.rename(columns={'hybrid_score_y' : 'hybrid_score'}, inplace=True)
    ratings.drop_duplicates(subset=['user_id', 'anime_id'], inplace=True) 

    # Calculate MAE
    mae = mean_absolute_error(ratings['normalized_rating'], ratings['hybrid_score'])
    return mae

Range of MAE values will be dependant on the scale of ratings used, here it will be from 1-10.

The lower the MAE the better.

In [408]:
mae = calculate_mae(mae_df)
print("MAE:", mae)

MAE: 0.5747027733879503
